# fact_messaging_time

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
# Con el ultimo mensajero, granularidad por minuto y hora redondeada. Es decir, todas las horas son en punto. Por ejemplo, si en la base de datos un mensajero acepto un servicio a las 8:47 en la bodega se guarda con la hora 8:00

mensajeria_servicio = pd.read_sql_table('mensajeria_servicio', engineSource)
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', engineSource)
clientes_mensajeroaquitoy = pd.read_sql_table('clientes_mensajeroaquitoy', engineSource)

mensajeria_servicio.drop(columns=["descripcion", "nombre_solicitante", "fecha_solicitud", "hora_solicitud", "fecha_deseada", "hora_deseada", "nombre_recibe", "telefono_recibe", "descripcion_pago", "ida_y_regreso", "activo", "novedades", "cliente_id", "destino_id", "origen_id", "tipo_pago_id", "tipo_servicio_id", "tipo_vehiculo_id", "usuario_id", "prioridad", "ciudad_destino_id", "ciudad_origen_id", "hora_visto_por_mensajero", "visto_por_mensajero", "descripcion_multiples_origenes", "multiples_origenes", "asignar_mensajero", "es_prueba", "descripcion_cancelado"], inplace=True)
mensajeria_estadosservicio.drop(columns=["fecha", "foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)
clientes_mensajeroaquitoy.drop(columns=["activo", "fecha_entrada", "fecha_salida", "salario", "telefono", "ciudad_operacion_id", "token_Firebase", "url_foto"], inplace=True)

mensajeria_servicio['mensajero2_id'].fillna(mensajeria_servicio['mensajero_id'], inplace=True)
mensajeria_servicio['mensajero3_id'].fillna(mensajeria_servicio['mensajero2_id'], inplace=True)

mensajeria_estadosservicio = mensajeria_estadosservicio[mensajeria_estadosservicio['estado_id'] == 2]

fact_messaging_time_nm1 = pd.merge(mensajeria_servicio, mensajeria_estadosservicio, left_on='id', right_on='servicio_id', how='inner')
fact_messaging_time = pd.merge(fact_messaging_time_nm1, clientes_mensajeroaquitoy, left_on='mensajero3_id', right_on='id', how='inner')

registro_de_fact_messaging_time_con_tiempos_y_mensajeros = []
conteo_de_mensajeros_por_servicio = {}
for index, row in fact_messaging_time.iterrows():
    servicio_id = fact_messaging_time.iloc[index, 7]
    if servicio_id not in conteo_de_mensajeros_por_servicio:
        conteo_de_mensajeros_por_servicio[servicio_id] = 1
    elif conteo_de_mensajeros_por_servicio[servicio_id] == 1:
        conteo_de_mensajeros_por_servicio[servicio_id] += 1
    elif conteo_de_mensajeros_por_servicio[servicio_id] == 2:
        nueva_fila = {'time': fact_messaging_time.iloc[index, 5].replace(minute=0, second=0), 'key_messenger': int(fact_messaging_time.iloc[index, 9])}
        registro_de_fact_messaging_time_con_tiempos_y_mensajeros.append(nueva_fila)
        conteo_de_mensajeros_por_servicio[servicio_id] += 1

fact_messaging_time_con_tiempos_y_mensajeros = pd.DataFrame(registro_de_fact_messaging_time_con_tiempos_y_mensajeros)
fact_messaging_time_con_tiempos_y_mensajeros
fact_messaging_time_con_tiempos_y_mensajeros_agrupados = fact_messaging_time_con_tiempos_y_mensajeros.groupby(['key_messenger', 'time']).size().reset_index(name='total_time')
fact_messaging_time_con_tiempos_y_mensajeros_agrupados["key_fact_messaging_time"] = range(1, len(fact_messaging_time_con_tiempos_y_mensajeros_agrupados) + 1)

dim_time = pd.read_sql_table('dim_time', engineDestination)

fact_messaging_time_con_tiempos_y_mensajeros_agrupadoss_con_time_id = fact_messaging_time_con_tiempos_y_mensajeros_agrupados.merge(dim_time[['time', 'key_dim_time']], on='time', how='left')
fact_messaging_time_con_tiempos_y_mensajeros_agrupadoss_con_time_id.drop(columns=['time'], inplace=True)
fact_messaging_time_con_tiempos_y_mensajeros_agrupadoss_con_time_id.rename(columns={'key_dim_time': 'key_time'}, inplace=True)

fact_messaging_time_con_tiempos_y_mensajeros_agrupadoss_con_time_id

### Carga de datos

In [ ]:
fact_messaging_time_con_tiempos_y_mensajeros_agrupadoss_con_time_id.to_sql("fact_messaging_time", engineDestination, index=False, if_exists="replace")